In [1]:
from keras.models import Sequential
from keras.layers import SimpleRNN

Using TensorFlow backend.


In [2]:
model = Sequential()
model.add(SimpleRNN(3, input_shape=(2,10))) # RNN층 추가
# model.add(SimpleRNN(3, input_length=2, input_dim=10))와 동일함
model.summary()

W0821 09:24:11.576261  6364 deprecation_wrapper.py:119] From C:\Users\Affinity\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0821 09:24:11.630116  6364 deprecation_wrapper.py:119] From C:\Users\Affinity\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0821 09:24:11.645078  6364 deprecation_wrapper.py:119] From C:\Users\Affinity\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_1 (SimpleRNN)     (None, 3)                 42        
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


### RNN 구현하기 (케라스 이용)
#### batch_size를 미리 정의

In [5]:
model = Sequential()
model.add(SimpleRNN(3, batch_input_shape=(8,2,10), return_sequences=True)) # RNN층 추가
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_4 (SimpleRNN)     (8, 2, 3)                 42        
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


#### 출력값으로 batch_size, timesteps, output_dimm 크기의 3D 텐서를 리턴
#### return_sequences 매개변수에 True를 기재하여 3D 텐서를 리턴
#### 출력의 크기가 8,2,3

## RNN을 이용한 언어모델링

In [10]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import SimpleRNN
#tf.enable_eager_execution()

In [11]:
train_X = [[0.1, 4.2, 1.5, 1.1, 2.8], [1.0, 3.1, 2.5, 0.7, 1.1], [0.3, 2.1, 1.5, 2.1, 0.1], [2.2, 1.4, 0.5, 0.9, 1.1]]
print(np.shape(train_X))

(4, 5)


In [15]:
# 위의 2D 텐서를 3D 텐서로 변경
train_X = [[[0.1, 4.2, 1.5, 1.1, 2.8], [1.0, 3.1, 2.5, 0.7, 1.1], [0.3, 2.1, 1.5, 2.1, 0.1], [2.2, 1.4, 0.5, 0.9, 1.1]]]
train_X = np.array(train_X, dtype=np.float32)
print(train_X.shape)

(1, 4, 5)


In [16]:
rnn = SimpleRNN(3, return_sequences=True, return_state=True)
hidden_states, last_states = rnn(train_X)

print('train_X: {}, shape: {}'.format(train_X, train_X.shape)) # 입력으로 사용한 훈련 데이터
print('hidden states  {}, shape : {}'. format(hidden_states, hidden_states.shape))# 모든 time-step의 은닉 상태
print('last hidden state : {}, shape: {}'.format(last_states, last_states.shape)) # 마지막 은닉 상태

train_X: [[[0.1 4.2 1.5 1.1 2.8]
  [1.  3.1 2.5 0.7 1.1]
  [0.3 2.1 1.5 2.1 0.1]
  [2.2 1.4 0.5 0.9 1.1]]], shape: (1, 4, 5)
hidden states  Tensor("simple_rnn_1_1/transpose_1:0", shape=(1, 4, 3), dtype=float32), shape : (1, 4, 3)
last hidden state : Tensor("simple_rnn_1_1/while/Exit_3:0", shape=(1, 3), dtype=float32), shape: (1, 3)


### 다음 단어 예측하기: RNN을 통해서 주어진 문장을 학습하고, 다음 단어를 예측하는 인공 신경망 모델 만들기

In [17]:
text="나랑 점심 먹으러 갈래 메뉴는 햄버거 점심 메뉴 좋지"
# 토큰화와 정수 인코딩
from keras_preprocessing.text import Tokenizer
t = Tokenizer()
t.fit_on_texts([text])
encoded = t.texts_to_sequences([text])[0]
# [0]을 해주지 않으면 [[contents]]와 같은 리스트 안의 리스트 형태로 저장 됨.
# [0]을 해주면 [contents]와 같은 하나의 리스트로 저장됨.

In [18]:
vocab_size = len(t.word_index) + 1

print('단어 집합의 크기: %d' % vocab_size)

단어 집합의 크기: 9


In [19]:
print(t.word_index)

{'점심': 1, '나랑': 2, '먹으러': 3, '갈래': 4, '메뉴는': 5, '햄버거': 6, '메뉴': 7, '좋지': 8}


In [20]:
sequences = list()
for c in range(1, len(encoded)):
    sequence = encoded[c-1:c+1]
    sequences.append(sequence)
print('단어 묶음의 개수: %d' % len(sequences))

단어 묶음의 개수: 8


In [21]:
print(sequences)

[[2, 1], [1, 3], [3, 4], [4, 5], [5, 6], [6, 1], [1, 7], [7, 8]]


In [35]:
import numpy as np
X,y = zip(*sequences)
X=np.array(X)
y = np.array(y)

In [36]:
print(X)
print(y)

[2 1 3 4 5 6 1 7]
[1 3 4 5 6 1 7 8]


In [37]:
from keras.utils import to_categorical
y=to_categorical(y, num_classes = vocab_size)
print(y)

[[0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [24]:
from keras.layers import Embedding, Dense, SimpleRNN
from keras.models import Sequential
tf.compat.v1.disable_eager_execution()

In [38]:
model=Sequential()
model.add(Embedding(vocab_size, 9, input_length=1))
model.add(SimpleRNN(9))
model.add(Dense(vocab_size, activation='softmax'))

In [39]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 1, 9)              81        
_________________________________________________________________
simple_rnn_6 (SimpleRNN)     (None, 9)                 171       
_________________________________________________________________
dense_2 (Dense)              (None, 9)                 90        
Total params: 342
Trainable params: 342
Non-trainable params: 0
_________________________________________________________________


In [26]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=500, verbose=2)

W0821 11:14:37.699520  6364 deprecation_wrapper.py:119] From C:\Users\Affinity\Anaconda3\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0821 11:14:37.721451  6364 deprecation_wrapper.py:119] From C:\Users\Affinity\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.

W0821 11:14:37.809216  6364 deprecation.py:323] From C:\Users\Affinity\Anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0821 11:14:37.941862  6364 deprecation_wrapper.py:119] From C:\Users\Affinity\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use

Epoch 1/500
 - 1s - loss: 2.2111 - acc: 0.0000e+00
Epoch 2/500
 - 0s - loss: 2.2076 - acc: 0.0000e+00
Epoch 3/500
 - 0s - loss: 2.2041 - acc: 0.1250
Epoch 4/500
 - 0s - loss: 2.2007 - acc: 0.1250
Epoch 5/500
 - 0s - loss: 2.1973 - acc: 0.1250
Epoch 6/500
 - 0s - loss: 2.1939 - acc: 0.1250
Epoch 7/500
 - 0s - loss: 2.1905 - acc: 0.1250
Epoch 8/500
 - 0s - loss: 2.1871 - acc: 0.1250
Epoch 9/500
 - 0s - loss: 2.1838 - acc: 0.2500
Epoch 10/500
 - 0s - loss: 2.1804 - acc: 0.2500
Epoch 11/500
 - 0s - loss: 2.1771 - acc: 0.2500
Epoch 12/500
 - 0s - loss: 2.1737 - acc: 0.2500
Epoch 13/500
 - 0s - loss: 2.1704 - acc: 0.2500
Epoch 14/500
 - 0s - loss: 2.1671 - acc: 0.2500
Epoch 15/500
 - 0s - loss: 2.1638 - acc: 0.2500
Epoch 16/500
 - 0s - loss: 2.1604 - acc: 0.2500
Epoch 17/500
 - 0s - loss: 2.1571 - acc: 0.2500
Epoch 18/500
 - 0s - loss: 2.1538 - acc: 0.2500
Epoch 19/500
 - 0s - loss: 2.1505 - acc: 0.3750
Epoch 20/500
 - 0s - loss: 2.1471 - acc: 0.3750
Epoch 21/500
 - 0s - loss: 2.1438 - acc: 

Epoch 171/500
 - 0s - loss: 1.1493 - acc: 0.7500
Epoch 172/500
 - 0s - loss: 1.1416 - acc: 0.7500
Epoch 173/500
 - 0s - loss: 1.1339 - acc: 0.7500
Epoch 174/500
 - 0s - loss: 1.1263 - acc: 0.7500
Epoch 175/500
 - 0s - loss: 1.1186 - acc: 0.7500
Epoch 176/500
 - 0s - loss: 1.1110 - acc: 0.7500
Epoch 177/500
 - 0s - loss: 1.1035 - acc: 0.7500
Epoch 178/500
 - 0s - loss: 1.0960 - acc: 0.7500
Epoch 179/500
 - 0s - loss: 1.0885 - acc: 0.7500
Epoch 180/500
 - 0s - loss: 1.0811 - acc: 0.7500
Epoch 181/500
 - 0s - loss: 1.0737 - acc: 0.7500
Epoch 182/500
 - 0s - loss: 1.0664 - acc: 0.7500
Epoch 183/500
 - 0s - loss: 1.0591 - acc: 0.7500
Epoch 184/500
 - 0s - loss: 1.0518 - acc: 0.7500
Epoch 185/500
 - 0s - loss: 1.0446 - acc: 0.7500
Epoch 186/500
 - 0s - loss: 1.0375 - acc: 0.7500
Epoch 187/500
 - 0s - loss: 1.0303 - acc: 0.7500
Epoch 188/500
 - 0s - loss: 1.0232 - acc: 0.7500
Epoch 189/500
 - 0s - loss: 1.0162 - acc: 0.7500
Epoch 190/500
 - 0s - loss: 1.0092 - acc: 0.7500
Epoch 191/500
 - 0s 

Epoch 339/500
 - 0s - loss: 0.4097 - acc: 0.8750
Epoch 340/500
 - 0s - loss: 0.4080 - acc: 0.8750
Epoch 341/500
 - 0s - loss: 0.4062 - acc: 0.8750
Epoch 342/500
 - 0s - loss: 0.4045 - acc: 0.8750
Epoch 343/500
 - 0s - loss: 0.4027 - acc: 0.8750
Epoch 344/500
 - 0s - loss: 0.4010 - acc: 0.8750
Epoch 345/500
 - 0s - loss: 0.3993 - acc: 0.8750
Epoch 346/500
 - 0s - loss: 0.3977 - acc: 0.8750
Epoch 347/500
 - 0s - loss: 0.3960 - acc: 0.8750
Epoch 348/500
 - 0s - loss: 0.3944 - acc: 0.8750
Epoch 349/500
 - 0s - loss: 0.3927 - acc: 0.8750
Epoch 350/500
 - 0s - loss: 0.3911 - acc: 0.8750
Epoch 351/500
 - 0s - loss: 0.3895 - acc: 0.8750
Epoch 352/500
 - 0s - loss: 0.3880 - acc: 0.8750
Epoch 353/500
 - 0s - loss: 0.3864 - acc: 0.8750
Epoch 354/500
 - 0s - loss: 0.3849 - acc: 0.8750
Epoch 355/500
 - 0s - loss: 0.3833 - acc: 0.8750
Epoch 356/500
 - 0s - loss: 0.3818 - acc: 0.8750
Epoch 357/500
 - 0s - loss: 0.3803 - acc: 0.8750
Epoch 358/500
 - 0s - loss: 0.3788 - acc: 0.8750
Epoch 359/500
 - 0s 

In [27]:
print(t.word_index.items())

dict_items([('점심', 1), ('나랑', 2), ('먹으러', 3), ('갈래', 4), ('메뉴는', 5), ('햄버거', 6), ('메뉴', 7), ('좋지', 8)])


In [28]:
def predict_next_word(model, t, current_word): # 모델, 토크나이저, 현재 단어를 받아온다.
    encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
    encoded = np.array(encoded) # 현재 단어에 대한 정수 인코딩
    result = model.predict_classes(encoded, verbose=0)
    # 입력한 x(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장.
    for word, index in t.word_index.items():
        if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
            return word # 그 단어를 출력

In [29]:
print(predict_next_word(model, t, "먹으러"))

갈래


In [32]:
def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 출력하기 위해 저장
    sentence = ''
    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0]
        encoded = np.array(encoded)
        result = model.predict_classes(encoded, verbose=0)
        
        for word, index in t.word_index.items():
            if index == result:
                break # 해당 단어가 예측 단어이므로 break
        current_word = word
        sentence = sentence + ' ' + word
    sentence = init_word + sentence
    return sentence

In [33]:
print(sentence_generation(model, t, '먹으러', 6))

먹으러 갈래 메뉴는 햄버거 점심 메뉴 좋지


### RNN을 이용한 언어 모델링 (2)

In [43]:
text="""경마장에 있는 말이 뛰고 있다\n
그의 말이 법이다\n
가는 말이 고와야 오는 말이 곱다\n"""

In [44]:
from keras_preprocessing.text import Tokenizer
t = Tokenizer()
t.fit_on_texts([text])
encoded = t.texts_to_sequences([text])[0]

In [45]:
vocab_size = len(t.word_index)+1
## 케라스 토크나이저의 정수 인코딩은 인덱스가 1부터 시작하지만, 
# 케라스 원핫 인코딩에서 배열의 인덱스가 0부터 시작하기 때문에
# 배열의 크기를 실제 단어 집합의 크기보다 +1로 생성해야하므로 미리 +1 선언
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 12


In [46]:
sequences = list()
for line in text.split('\n'):
    encoded = t.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)
        
print('훈련 데이터의 개수: %d' % len(sequences))

훈련 데이터의 개수: 11


In [47]:
print(sequences)

[[2, 3], [2, 3, 1], [2, 3, 1, 4], [2, 3, 1, 4, 5], [6, 1], [6, 1, 7], [8, 1], [8, 1, 9], [8, 1, 9, 10], [8, 1, 9, 10, 1], [8, 1, 9, 10, 1, 11]]


In [48]:
print(max(len(l) for l in sequences)) # 모든 데이터에서 길이가 가장 긴 데이터의 길이 출력

6


In [49]:
from keras.preprocessing.sequence import pad_sequences
sequences = pad_sequences(sequences, maxlen = 6, padding = 'pre')

In [50]:
print(sequences)

[[ 0  0  0  0  2  3]
 [ 0  0  0  2  3  1]
 [ 0  0  2  3  1  4]
 [ 0  2  3  1  4  5]
 [ 0  0  0  0  6  1]
 [ 0  0  0  6  1  7]
 [ 0  0  0  0  8  1]
 [ 0  0  0  8  1  9]
 [ 0  0  8  1  9 10]
 [ 0  8  1  9 10  1]
 [ 8  1  9 10  1 11]]


In [52]:
import numpy as np
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]
# 리스트의 마지막 열을 제외하고 저장한 것은 X
# 리스트의 마지막 열만 저장한 것은 y

In [53]:
print(X)
print(y)

[[ 0  0  0  0  2]
 [ 0  0  0  2  3]
 [ 0  0  2  3  1]
 [ 0  2  3  1  4]
 [ 0  0  0  0  6]
 [ 0  0  0  6  1]
 [ 0  0  0  0  8]
 [ 0  0  0  8  1]
 [ 0  0  8  1  9]
 [ 0  8  1  9 10]
 [ 8  1  9 10  1]]
[ 3  1  4  5  1  7  1  9 10  1 11]


In [54]:
from keras.utils import to_categorical
y = to_categorical(y, num_classes = vocab_size)
print(y)

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [55]:
from keras.layers import Embedding, Dense, SimpleRNN
from keras.models import Sequential

model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=5)) # y를 제거하였으므로 이제 X의 길이는 5
model.add(SimpleRNN(32))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X,y,epochs=200, verbose=2)

Epoch 1/200
 - 0s - loss: 2.5328 - acc: 0.0000e+00
Epoch 2/200
 - 0s - loss: 2.5191 - acc: 0.0000e+00
Epoch 3/200
 - 0s - loss: 2.5059 - acc: 0.0000e+00
Epoch 4/200
 - 0s - loss: 2.4931 - acc: 0.0000e+00
Epoch 5/200
 - 0s - loss: 2.4806 - acc: 0.0909
Epoch 6/200
 - 0s - loss: 2.4684 - acc: 0.1818
Epoch 7/200
 - 0s - loss: 2.4563 - acc: 0.2727
Epoch 8/200
 - 0s - loss: 2.4441 - acc: 0.2727
Epoch 9/200
 - 0s - loss: 2.4319 - acc: 0.3636
Epoch 10/200
 - 0s - loss: 2.4195 - acc: 0.3636
Epoch 11/200
 - 0s - loss: 2.4068 - acc: 0.3636
Epoch 12/200
 - 0s - loss: 2.3938 - acc: 0.3636
Epoch 13/200
 - 0s - loss: 2.3804 - acc: 0.3636
Epoch 14/200
 - 0s - loss: 2.3665 - acc: 0.3636
Epoch 15/200
 - 0s - loss: 2.3520 - acc: 0.3636
Epoch 16/200
 - 0s - loss: 2.3369 - acc: 0.3636
Epoch 17/200
 - 0s - loss: 2.3211 - acc: 0.3636
Epoch 18/200
 - 0s - loss: 2.3046 - acc: 0.3636
Epoch 19/200
 - 0s - loss: 2.2872 - acc: 0.3636
Epoch 20/200
 - 0s - loss: 2.2691 - acc: 0.3636
Epoch 21/200
 - 0s - loss: 2.2502

Epoch 171/200
 - 0s - loss: 0.1953 - acc: 1.0000
Epoch 172/200
 - 0s - loss: 0.1919 - acc: 1.0000
Epoch 173/200
 - 0s - loss: 0.1886 - acc: 1.0000
Epoch 174/200
 - 0s - loss: 0.1853 - acc: 1.0000
Epoch 175/200
 - 0s - loss: 0.1821 - acc: 1.0000
Epoch 176/200
 - 0s - loss: 0.1789 - acc: 1.0000
Epoch 177/200
 - 0s - loss: 0.1757 - acc: 1.0000
Epoch 178/200
 - 0s - loss: 0.1726 - acc: 1.0000
Epoch 179/200
 - 0s - loss: 0.1695 - acc: 1.0000
Epoch 180/200
 - 0s - loss: 0.1665 - acc: 1.0000
Epoch 181/200
 - 0s - loss: 0.1635 - acc: 1.0000
Epoch 182/200
 - 0s - loss: 0.1605 - acc: 1.0000
Epoch 183/200
 - 0s - loss: 0.1576 - acc: 1.0000
Epoch 184/200
 - 0s - loss: 0.1548 - acc: 1.0000
Epoch 185/200
 - 0s - loss: 0.1520 - acc: 1.0000
Epoch 186/200
 - 0s - loss: 0.1492 - acc: 1.0000
Epoch 187/200
 - 0s - loss: 0.1465 - acc: 1.0000
Epoch 188/200
 - 0s - loss: 0.1438 - acc: 1.0000
Epoch 189/200
 - 0s - loss: 0.1412 - acc: 1.0000
Epoch 190/200
 - 0s - loss: 0.1386 - acc: 1.0000
Epoch 191/200
 - 0s 

In [56]:
def sentence_generation(model, t, current_word, n): #모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기 위해 저장
    sentence=''
    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen = 5, padding = 'pre') # 데이터에 대한 패딩
        result = model.predict_classes(encoded, verbose=0)
    #입력한 x (현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 RESULT에 저장. 
        for word, index in t.word_index.items():
            if index == result: # 만약 예측한 단어와 인덱스가 동일한 단어가 있다면
                break
        current_word = current_word + ' ' + word 
        sentence = sentence + ' ' + word
    sentence = init_word + sentence
    return sentence

In [57]:
print(sentence_generation(model, t, '경마장에', 4))

경마장에 있는 말이 뛰고 있다


In [58]:
print(sentence_generation(model, t, '그의', 2))

그의 말이 법이다


In [59]:
print(sentence_generation(model, t, '가는', 5))

가는 말이 고와야 오는 말이 곱다


### LSTM (Long Short-Term Memory, 장단기 메모리)

In [60]:
import pandas as pd
df = pd.read_csv('ArticlesApril2018.csv')
df.head()

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleaders’ Settlement Offer: ...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"“I understand that they could meet with us, pa...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...
2,5adf4626068401528a2aa628,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,What’s it like to eat at the second incarnatio...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...
3,5adf40d2068401528a2aa619,626,By JULIE HIRSCHFELD DAVIS and PETER BAKER,article,Unknown,"['Macron, Emmanuel (1977- )', 'Trump, Donald J...",68,Washington,0,2018-04-24 14:35:57,Europe,President Trump welcomed President Emmanuel Ma...,The New York Times,News,https://www.nytimes.com/2018/04/24/world/europ...
4,5adf3d64068401528a2aa60f,815,By IAN AUSTEN and DAN BILEFSKY,article,Unknown,"['Toronto, Ontario, Attack (April, 2018)', 'Mu...",68,Foreign,0,2018-04-24 14:21:21,Canada,"Alek Minassian, 25, a resident of Toronto’s Ri...",The New York Times,News,https://www.nytimes.com/2018/04/24/world/canad...


In [61]:
print(df.columns)

Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')


In [62]:
print('열의 개수: ', len(df.columns))

열의 개수:  15


In [64]:
df['headline'].isnull().values.any()

False

In [65]:
headline = []
headline.extend(list(df.headline.values))
headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown']

In [66]:
len(headline)

1324

In [67]:
headline = [n for n in headline if n != "Unknown"] # Unknown 데이터 제거 
len(headline)

1214

In [68]:
headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'How a Bag of Texas Dirt  Became a Times Tradition',
 'Is School a Place for Self-Expression?']

In [69]:
from string import punctuation 
def repreprocessing(s):
    s=s.encode("utf8").decode("ascii", "ignore") # 코드 추가
    return ''.join(c for c in s if s not in punctuation).lower() # 구두점 제거와 동시에 소문자화
text = [repreprocessing(x) for x in headline]
text[:5]

['former n.f.l. cheerleaders settlement offer: $1 and a meeting with goodell',
 'e.p.a. to unveil a new rule. its effect: less science in policymaking.',
 'the new noma, explained',
 'how a bag of texas dirt  became a times tradition',
 'is school a place for self-expression?']

In [72]:
from keras_preprocessing.text import Tokenizer
t = Tokenizer()
t.fit_on_texts(text)
vocab_size = len(t.word_index) + 1
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 3474


In [73]:
sequences = list()

for line in text: # 1214개의 데이터에 대해서 데이터를 1개씩 가져온다. 
    encoded = t.texts_to_sequences([line])[0] # 각 데이터에 대한 정수 인코딩
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)
sequences[:11]

[[107, 178],
 [107, 178, 92],
 [107, 178, 92, 146],
 [107, 178, 92, 146, 392],
 [107, 178, 92, 146, 392, 1144],
 [107, 178, 92, 146, 392, 1144, 601],
 [107, 178, 92, 146, 392, 1144, 601, 54],
 [107, 178, 92, 146, 392, 1144, 601, 54, 7],
 [107, 178, 92, 146, 392, 1144, 601, 54, 7, 2],
 [107, 178, 92, 146, 392, 1144, 601, 54, 7, 2, 393],
 [107, 178, 92, 146, 392, 1144, 601, 54, 7, 2, 393, 10]]

In [74]:
index_to_word={}
for key, value in t.word_index.items(): # 인덱스를 단어로 바꾸기 위해 index_to_word를 생성
    index_to_word[value] = key
index_to_word[582]

'mega'

In [75]:
max_len=max(len(l) for l in sequences)
print(max_len)

28


In [76]:
from keras.preprocessing.sequence import pad_sequences
sequences = pad_sequences(sequences, maxlen = max_len, padding='pre')
print(sequences[:3])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0 107 178]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0 107 178  92]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0 107 178  92 146]]


#### X데이터와 y데이터 분리

In [78]:
import numpy as np
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]
print(X[:3])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0 107]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0 107 178]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0 107 178  92]]


In [79]:
print(y[:3])

[178  92 146]


In [80]:
from keras.utils import to_categorical
y = to_categorical(y, num_classes=vocab_size)

In [81]:
from keras.layers import Embedding, Dense, LSTM
from keras.models import Sequential

model = Sequential()
model.add(Embedding(vocab_size, 10, input_length = max_len-1))
# y 데이터를 분리하였으므로 이제 x데이터의 길이는 기존 데이터의 길이 -1
model.add(LSTM(128))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X,y,epochs=200, verbose=2)

Epoch 1/200
 - 6s - loss: 7.6021 - acc: 0.0320
Epoch 2/200
 - 6s - loss: 7.0957 - acc: 0.0293
Epoch 3/200
 - 7s - loss: 6.9586 - acc: 0.0347
Epoch 4/200
 - 6s - loss: 6.8194 - acc: 0.0406
Epoch 5/200
 - 6s - loss: 6.6474 - acc: 0.0424
Epoch 6/200
 - 7s - loss: 6.4560 - acc: 0.0483
Epoch 7/200
 - 7s - loss: 6.2552 - acc: 0.0521
Epoch 8/200
 - 7s - loss: 6.0493 - acc: 0.0605
Epoch 9/200
 - 7s - loss: 5.8465 - acc: 0.0635
Epoch 10/200
 - 7s - loss: 5.6500 - acc: 0.0687
Epoch 11/200
 - 7s - loss: 5.4594 - acc: 0.0743
Epoch 12/200
 - 7s - loss: 5.2783 - acc: 0.0819
Epoch 13/200
 - 7s - loss: 5.1073 - acc: 0.0875
Epoch 14/200
 - 7s - loss: 4.9449 - acc: 0.0956
Epoch 15/200
 - 7s - loss: 4.7859 - acc: 0.1108
Epoch 16/200
 - 7s - loss: 4.6389 - acc: 0.1262
Epoch 17/200
 - 7s - loss: 4.4917 - acc: 0.1459
Epoch 18/200
 - 7s - loss: 4.3529 - acc: 0.1628
Epoch 19/200
 - 7s - loss: 4.2195 - acc: 0.1815
Epoch 20/200
 - 7s - loss: 4.0895 - acc: 0.2026
Epoch 21/200
 - 7s - loss: 3.9637 - acc: 0.2212
E

Epoch 171/200
 - 7s - loss: 0.2891 - acc: 0.9182
Epoch 172/200
 - 7s - loss: 0.2954 - acc: 0.9155
Epoch 173/200
 - 7s - loss: 0.3693 - acc: 0.9004
Epoch 174/200
 - 7s - loss: 0.3102 - acc: 0.9142
Epoch 175/200
 - 7s - loss: 0.2857 - acc: 0.9176
Epoch 176/200
 - 7s - loss: 0.2824 - acc: 0.9182
Epoch 177/200
 - 7s - loss: 0.2795 - acc: 0.9191
Epoch 178/200
 - 7s - loss: 0.2794 - acc: 0.9185
Epoch 179/200
 - 7s - loss: 0.2782 - acc: 0.9180
Epoch 180/200
 - 7s - loss: 0.2786 - acc: 0.9166
Epoch 181/200
 - 7s - loss: 0.2786 - acc: 0.9181
Epoch 182/200
 - 7s - loss: 0.2760 - acc: 0.9190
Epoch 183/200
 - 7s - loss: 0.2771 - acc: 0.9180
Epoch 184/200
 - 7s - loss: 0.2757 - acc: 0.9187
Epoch 185/200
 - 7s - loss: 0.2764 - acc: 0.9173
Epoch 186/200
 - 7s - loss: 0.2768 - acc: 0.9181
Epoch 187/200
 - 7s - loss: 0.2759 - acc: 0.9181
Epoch 188/200
 - 7s - loss: 0.2766 - acc: 0.9178
Epoch 189/200
 - 7s - loss: 0.2755 - acc: 0.9178
Epoch 190/200
 - 7s - loss: 0.2831 - acc: 0.9160
Epoch 191/200
 - 7s 